In [3]:
import numpy as np

In [4]:
# Environment is set up under the assumption that the arrival An is Poisson
class Environment:
    def __init__(self, poisson_lambda, queue_max, service_mu, service_lambda, cost_coeff, initial_queuelength):
        self.poisson_lambda = poisson_lambda
        self.queue_max = queue_max
        self.service_mu = service_mu
        self.service_lambda = service_lambda
        self.cost_coeff = cost_coeff
        self.initial_queuelength = initial_queuelength
        self.queuelength = self.initial_queuelength
    
    def get_current_state(self):
        return self.queuelength
    
    def simulate_and_observe(self, action):
        arrivals = np.random.poisson(self.poisson_lambda)
        self.queuelength = self.queuelength + arrivals
        reward = self.queuelength
        
        if self.queuelength > 20:
            self.queuelength = 20
            
        if self.queuelength > 0:
            service = 0
            if action == 1:
                randp = self.service_mu
            if action == 2:
                randp = self.service_lambda
                    
            if np.random.rand <= randp:
                service = 1
            self.queuelength = self.queuelength - service
        
        reward = reward + self.cost_coeff * action * action;
        
        return [self.queuelength, -reward]

In [ ]:
# Every Visit Monte carlo estimate of a policy - action = 1 if state in [0,10], action = 2 if state in [11, 20]
num_episodes = 2000
horizon = 100
state_space = range(21)
discount_factor = 0.2
step_size = 0.01
trace_lambda = 0.5

every_visit_estimate = np.zeros([len(state_space),1])
every_visit_count = np.zeros([len(state_space),1])
eligibility_trace = np.zeros([len(state_space), 1])

for e in range(num_episodes):
    
    environment = Environment(0.3, 20, 0.2, 0.7, 1, 0)
    state_sequence = []
    reward_sequence = []

    for t in range(horizon):
        state = environment.get_current_state()
        state_sequence.append(state)
        if state <= 10:
            action = 1
        else:
            action = 2
        [state, reward] = environment.simulate_and_observe(action)
        reward_sequence.append(reward)
    # print state_sequence
    # print reward_sequence
    
    for t in range(horizon - 1):
        s = state_sequence[t]
        every_visit_return = reward_sequence[t] + discount_factor * every_visit_estimate[state_sequence[t + 1]]
        delta = step_size * (every_visit_return - every_visit_estimate[s])
        eligibility_trace[s] = eligibility_trace[s] + 1
        
        for s in state_space:
            every_visit_estimate[s] = every_visit_estimate[s] + eligibility_trace[s] * delta
            eligibility_trace[s] = eligibility_trace[s] * discount_factor * trace_lambda


In [4]:
every_visit_estimate

array([[ -1.84308831],
       [ -2.93257329],
       [ -4.25364065],
       [ -5.51494439],
       [ -6.74546393],
       [ -7.93390801],
       [ -9.17628689],
       [-10.4371287 ],
       [-11.66445201],
       [-13.05770758],
       [-14.40772725],
       [-19.16412754],
       [-20.42208146],
       [-21.73539989],
       [-22.91806934],
       [-24.16695958],
       [-25.44977803],
       [-26.73241584],
       [-27.89832594],
       [-29.19255791],
       [-30.44099769]])